In [9]:
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel
import ftfy

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', lambda x: f'{x:.3f}')

import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import re

In [10]:
def ngrams(string, n=3):
    
    # string = re.sub(r'[,-./]|\sBD',r'', string)
    # ngrams = zip(*[string[i:] for i in range(n)])
    # return [''.join(ngram) for ngram in ngrams]
    
    
    string = ftfy.fix_text(string)
    string = string.encode("ascii", errors="ignore").decode()
    string = string.lower()
    chars_to_remove = [")","(",">","|","{","}","{","}","'"]
    rx = '[' + re.escape(''.join(chars_to_remove)) + ']'
    string = re.sub(rx, '', string)
    string = string.replace('&', 'and')
    string = string.replace(',',' ')
    string = string.replace('-', ' ')
    string = string.replace("isd", "intermediate school district")
    string = string.replace("usd", "unified school district")
    string = string.replace("sd", "school district")
    string = string.replace("schools", "")
    string = string.replace("school", "")
    string = string.replace("public", "")    
    string = string.replace("districts", "")
    string = string.replace("district", "")
    string = string.replace("independent", "")
    string = string.replace("unified", "")
    string = string.replace("metropolitan", "")
    string = string.replace("academy", "")
    string = string.replace("department", "")
    string = string.replace("board", "")
    string = string.replace("education", "")
    
    

    
    string = string.title()
    string = re.sub(' +',' ',string).strip()
    string = ' '+ string + ' '
    string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

In [11]:
# read in the dataset with goal data?? (training data)
account_path = None #insert path to account file here
clean_org_names = pd.read_csv(account_path)
clean_org_names = clean_org_names.iloc[:, 0:4]
# insert column name of column with training values
org_name_clean = clean_org_names['Account Name']#.unique()


print('Vecorizing the data - this could take a few minutes for large datasets...')
vectorizer = TfidfVectorizer(min_df=1, lowercase=False, analyzer=ngrams)
tfidf = vectorizer.fit_transform(org_name_clean)
print('Vecorizing completed...')

contact_path = None #insert path to contact file here
# read in the data to be acted upon (forward pass)
df_c = pd.read_excel(contact_path)
# enter your column header for the subset
df_c.dropna(subset='School District/Organization',inplace=True)

Vecorizing the data - this could take a few minutes for large datasets...
Vecorizing completed...


In [12]:
from sklearn.neighbors import NearestNeighbors
nbrs = NearestNeighbors(n_neighbors=1, n_jobs=-1).fit(tfidf)
org_column = 'School District/Organization' #column to match against in the messy data
unique_org = df_c[org_column].values # set used for increased performance:

### matching query
def getNearestN(query):
    queryTFIDF_ = vectorizer.transform(query)
    distances, indices = nbrs.kneighbors(queryTFIDF_)
    return distances, indices

import time
t1 = time.time()
print('getting nearest n...')
distances, indices = getNearestN(unique_org)
t = time.time()-t1
print("COMPLETED IN:", t)
unique_org = list(unique_org) #need to convert back to a list

print('finding matches...')
matches = []
for i,j in enumerate(indices):
    temp = [df_c['Contact ID'].values[i],round(1-distances[i][0],3), clean_org_names.values[j][0][0], clean_org_names.values[j][0][3],unique_org[i]]
    matches.append(temp)
print('Building data frame...')  
matches = pd.DataFrame(matches, columns=['Contact ID','accuracy','Record Id','Account Match','Contacts'])
print('Done')

getting nearest n...
COMPLETED IN: 0.11484098434448242
finding matches...
Building data frame...
Done


In [13]:
print("\'wrong\' count: " + str(len(matches.query('accuracy < .3').sort_values("accuracy", ascending=False))))

print("accuracy: "+ str(round((1-len(matches.query('accuracy < .3').sort_values("accuracy", ascending=False))/len(matches))*100, 2))+"%")

'wrong' count: 301
accuracy: 59.49%


In [14]:
a = np.array(matches['Record Id'].values.tolist())
matches['Record Id'] = np.where(matches.accuracy < .3, np.nan, a)
matches['Record Id'] = matches['Record Id'].replace("nan", np.nan)

b = np.array(matches['Account Match'].values.tolist())
matches['Account Match'] = np.where(matches.accuracy < .3, np.nan, b)
matches['Account Match'] = matches['Account Match'].replace("nan", np.nan)

In [15]:
matches.sort_values('accuracy', ascending = False).head(10)

,Contact ID,accuracy,Record Id,Account Match,Contacts
0,zcrm_5090086000000528052,1.000,zcrm_5090086000002814211,Jefferson County,Jefferson County Public Schools
358,zcrm_5090086000001583599,1.000,zcrm_5090086000002817053,Pitt County Schools,PITT COUNTY SCHOOLS
408,zcrm_5090086000001675081,1.000,zcrm_5090086000002764616,Bethel School District,Bethel School District
407,zcrm_5090086000001675080,1.000,zcrm_5090086000002764566,Bellingham,Bellingham School District
406,zcrm_5090086000001675079,1.000,zcrm_5090086000002764563,BELLEVUE PUBLIC SCHOOLS,Bellevue School District
405,zcrm_5090086000001675078,1.000,zcrm_5090086000002764563,BELLEVUE PUBLIC SCHOOLS,Bellevue School District
404,zcrm_5090086000001675077,1.000,zcrm_5090086000002764520,Battle Ground School District,Battle Ground School District
403,zcrm_5090086000001675076,1.000,zcrm_5090086000002764474,Bainbridge Island School District,Bainbridge Island School District
402,zcrm_5090086000001675075,1.000,zcrm_5090086000002764238,Auburn,Auburn School District
400,zcrm_5090086000001675073,1.000,zcrm_5090086000002764157,Anacortes School District,Anacortes School District


### re-merging

In [ ]:
outMatch = matches.drop(['Contacts'], axis = 1)

In [ ]:
cols = outMatch.columns.tolist()

In [ ]:
cols = ['Contact ID', 'Record Id', 'accuracy', 'Account Match']

In [ ]:
outMatch = outMatch[cols]

In [ ]:
df_c = pd.read_excel(contact_path)

In [ ]:
df = pd.merge(left=df_c, right=outMatch, on='Contact ID', how='outer',suffixes=["", "_Accounts"])

In [ ]:
df.to_csv("Final Contact - Account Integration.csv")